# Azure ML Anomaly Endpoint (Minimal)

This notebook sketches a tiny scoring function that mirrors the local z-score logic. Use it as a starting point to deploy an Azure ML online endpoint later.

In [ ]:
def score(request_json):
    amount = request_json['amount']
    history = request_json.get('vendorHistorySummary', {})
    mean = history.get('mean', 0)
    std = history.get('std', 0) or 1e-6
    z_score = (amount - mean) / std

    if z_score > 3:
        severity = 'HIGH'
    elif z_score > 2:
        severity = 'MEDIUM'
    else:
        severity = 'LOW'

    return {
        'zScore': float(z_score),
        'anomalySeverity': severity,
    }

# Example
score({
    'vendorName': 'nimbus stationery',
    'amount': 30000,
    'vendorHistorySummary': {'mean': 10000, 'std': 1200}
})


## Deployment outline
1. Create an Azure ML workspace and compute target.
2. Add a `score.py` with the function above and package it into an environment.
3. Use `azure-ai-ml` to create a managed online endpoint and deployment.
4. Set `AML_ENDPOINT_URL` and `AML_API_KEY` in `server/.env`.

The backend will automatically call the endpoint when those variables are set.